In [1]:
import climepi  # noqa
from climepi import climdata, epimod
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
import holoviews as hv
import numpy as np
import scipy.stats
import xarray as xr

from opts import get_opts
from utils import get_data_base_path, export_sensitivity_figure, notebook_config


# Configure notebook
notebook_config()

# Get options
options = get_opts()
year_range = options["year_range"]
locations = [options["location_default"]] + options["locations_additional"]
no_locations = len(locations)
epi_model_names = [
    options["epi_model_name_default"],
    options["epi_model_name_additional"],
]
epi_model_species = [
    options["epi_model_species_default"],
    options["epi_model_species_additional"],
]
confidence_level = options["confidence_level"]
polyfit_degree = options["polyfit_degree"]
plot_opts_temp = options["plot_opts_temp"]
plot_opts_epi = options["plot_opts_epi"]
figure_dir = options["figure_dir"]

years_summary = [2040, 2060, 2080]

2024-11-21 17:19:00,478 [INFO]: logger.py(log:11) >> ====== WebDriver manager ======
2024-11-21 17:19:00,478 [INFO]: logger.py(log:11) >> ====== WebDriver manager ======
2024-11-21 17:19:00,863 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2024-11-21 17:19:00,863 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2024-11-21 17:19:00,926 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2024-11-21 17:19:00,926 [INFO]: logger.py(log:11) >> Get LATEST chromedriver version for google-chrome
2024-11-21 17:19:00,978 [INFO]: logger.py(log:11) >> Driver [/Users/user/.wdm/drivers/chromedriver/mac64/131.0.6778.85/chromedriver-mac-arm64/chromedriver] found in cache
2024-11-21 17:19:00,978 [INFO]: logger.py(log:11) >> Driver [/Users/user/.wdm/drivers/chromedriver/mac64/131.0.6778.85/chromedriver-mac-arm64/chromedriver] found in cache


In [2]:
def make_summary_plot(ds, data_var=None, **kwargs):
    # Get data arrays of overall mean and variance decomposition between internal
    # variability, model and scenario uncertainty
    da_mean = (
        ds.climepi.ensemble_stats(polyfit_degree=polyfit_degree)
        .sel(ensemble_stat="mean", drop=True)
        .mean(dim=["scenario", "model"])[data_var]
    )
    da_var_decomp = ds.climepi.variance_decomposition(polyfit_degree=polyfit_degree)[
        data_var
    ]
    # Estimate confidence interval and breakdown into different sources of uncertainty
    ds_plot = xr.Dataset()
    ds_plot["mean"] = da_mean
    z = scipy.stats.norm.ppf(0.5 + confidence_level / 200)
    da_std_internal = np.sqrt(da_var_decomp.sel(variance_type="internal", drop=True))
    ds_plot["internal_lower"] = da_mean - z * da_std_internal
    ds_plot["internal_upper"] = da_mean + z * da_std_internal
    da_std_internal_model = np.sqrt(
        da_var_decomp.sel(variance_type=["internal", "model"]).sum(dim="variance_type")
    )
    ds_plot["model_lower"] = da_mean - z * da_std_internal_model
    ds_plot["model_upper"] = da_mean + z * da_std_internal_model
    da_std_internal_model_scenario = np.sqrt(da_var_decomp.sum(dim="variance_type"))
    ds_plot["scenario_lower"] = da_mean - z * da_std_internal_model_scenario
    ds_plot["scenario_upper"] = da_mean + z * da_std_internal_model_scenario
    # Plot
    ds_plot = ds_plot.sel(time=ds.time.dt.year.isin(years_summary))
    ds_plot["time"] = ds_plot["time.year"].astype("str")
    source = ColumnDataSource(ds_plot.to_dataframe())
    colors = hv.Cycle().values
    p = figure(
        x_range=FactorRange(*source.data["location_time"]),
        frame_width=720,
        frame_height=180,
    )
    p.line(  # Dummy line to get legend
        x=[p.x_range.start],
        y=[p.y_range.start],
        line_color="black",
        line_width=2,
        legend_label="Mean",
    )
    for bottom, top, color, legend_label in (
        ("internal_lower", "internal_upper", colors[0], "Internal variability"),
        ("model_lower", "internal_lower", colors[1], "Model uncertainty"),
        ("internal_upper", "model_upper", colors[1], None),
        ("scenario_lower", "model_lower", colors[2], "Scenario uncertainty"),
        ("model_upper", "scenario_upper", colors[2], None),
    ):
        kwargs_vbar = {
            "x": "location_time",
            "source": source,
            "bottom": bottom,
            "top": top,
            "fill_color": color,
            # "line_width": 0,
            "width": 0.8,
        }
        if legend_label is not None:
            kwargs_vbar["legend_label"] = legend_label
        p.vbar(**kwargs_vbar)
    p.vbar(
        x="location_time",
        source=source,
        bottom="mean",
        top="mean",
        line_color="black",
        line_width=2,
        width=0.8,
    )
    # Formatting
    p.x_range.range_padding = 0.075
    p.x_range.group_padding = 0.5
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    if "ylim" in kwargs:
        p.y_range.start = kwargs["ylim"][0]
        p.y_range.end = kwargs["ylim"][1]
    if "yticks" in kwargs:
        p.yaxis.ticker = kwargs["yticks"]
    if "xlabel" in kwargs:
        p.xaxis.axis_label = kwargs["xlabel"]
    if "ylabel" in kwargs:
        p.yaxis.axis_label = kwargs["ylabel"]
    fontsize = plot_opts_temp["fontsize"]
    p.xaxis.axis_label_text_font_size = f"{fontsize['labels']}pt"
    p.yaxis.axis_label_text_font_size = f"{fontsize['labels']}pt"
    p.xaxis.major_label_text_font_size = f"{fontsize['ticks']}pt"
    p.xaxis.group_text_font_size = f"{fontsize['ticks']}pt"
    p.yaxis.major_label_text_font_size = f"{fontsize['ticks']}pt"
    p.legend.label_text_font_size = f"{fontsize['legend']}pt"
    p.legend.title_text_font_size = f"{fontsize['legend_title']}pt"
    p.legend.location = kwargs.get("legend_location", "top_left")
    p.legend.margin = 5
    p.legend.padding = 5
    return p

In [3]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset("isimip_cities", base_dir=get_data_base_path())
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
    .sel(location=locations)
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")

In [4]:
p_clim = make_summary_plot(
    ds_temp_yearly,
    data_var="temperature",
    ylim=(9, 21),
    yticks=np.arange(9, 22, 3),
    ylabel="Annual mean\ntemperature (°C)",
    legend_location="bottom_right",
)
show(p_clim)

In [5]:
# Epi variability - plume
panels_epi_plume = []
for epi_model_name, species_curr in zip(epi_model_names, epi_model_species):
    epi_model = epimod.get_example_model(epi_model_name)
    ds_months_suitable = ds_clim.climepi.run_epi_model(
        epi_model, return_months_suitable=True
    )
    p = make_summary_plot(
        ds_months_suitable,
        data_var="months_suitable",
        ylim=(0, 12),
        yticks=np.arange(0, 13, 2),
        ylabel=f"Months suitable\n({species_curr})",
    )
    show(p)
    panels_epi_plume.append(p)

In [10]:
import importlib
import utils

importlib.reload(utils)

from utils import export_sensitivity_figure

# Combine panels, ordered by location
panels = [p_clim] + panels_epi_plume
export_sensitivity_figure(panels)